In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.exceptions import InvalidFileException
import os

In [2]:
path_data = 'original_data/formularios_03_08/'

In [3]:
def get_files(path_data):
    
    walk = list(os.walk(path_data))[1:]
    parsed_files = []

    for caminho in walk:

        folder, _, files = caminho

        files = [os.path.join(folder, file) for file in files
                if 'xls' in file.split('.')[-1]]

        parsed_files.extend(files)
    
    return parsed_files

In [4]:
def pegar_orcamento(sheet):
    
    dados = {}
    i=0
    for linha in range(21, 25):
        i+=1
        dados[f'desc_ano_{i}'] = sheet[f'C{linha}'].value
        dados[f'investimento_ano_{i}'] = sheet[f'E{linha}'].value
        dados[f'custeio_ano_{i}'] = sheet[f'F{linha}'].value
        dados[f'info_complementares_ano_{i}'] = sheet[f'I{linha}'].value
        
    return dados

In [5]:
def pegar_demais_orgaos(sheet):
    
    aux = lambda x: str(x) if x else ''
    
    data = [
        aux(sheet['B8'].value),
        aux(sheet['B9'].value),
        aux(sheet['B10'].value)
    ]
    
    return '\n'.join(data)

In [6]:
def pegar_dados(sheet):
    
    dados = dict(
        secretaria = sheet['F4'].value,
        projeto = sheet['F5'].value,
        demais_orgaos_participantes = pegar_demais_orgaos(sheet),
        andamento = sheet['F14'].value,
        acao_futura = sheet['J14'].value,
        estagio_atual = sheet['C16'].value,
        inicio = sheet['F27'].value,
        termino = sheet['N27'].value,
        objetivo = sheet['C30'].value,
        beneficiarios = sheet['C35'].value,
        empregos_geados = sheet['C39'].value,
        resultados_e_impactos = sheet['C42'].value,
        
    )
    
    
    dados.update(pegar_orcamento(sheet))
    
    return dados

In [7]:
def limpar_texto(dados):
    
    limpo = {}
    for key, val in dados.items():
        if type(val) is str:
            val_limpo = val.strip()
            val_limpo = val_limpo.replace(u'\u200b', '')
            limpo[key] = val_limpo
        else:
            limpo[key] = val
    
    return limpo

In [8]:
def checar_sheet_de_dados(sheet):
    
    return sheet['B2'].value == "Formulário de Ações - Plano de Retomada Econômica pós-pandemia"

In [9]:
def get_data_from_all_sheets(file_path):
    
    wb = load_workbook(file_path)
    sheets = wb.sheetnames
    
    all_sheet_data = []
    for sheet_name in sheets:
        sheet = wb[sheet_name]
        if checar_sheet_de_dados(sheet):
            data = pegar_dados(sheet)
            data = limpar_texto(data)
            all_sheet_data.append(data)
        else:
            print(f'File {file_path} contem planilha fora do padrão: {sheet_name}')
    
    return all_sheet_data

In [10]:
def get_all_data(path_data):
    
    files = get_files(path_data)
    data = []
    
    for file in files:
        try:
            data.extend(get_data_from_all_sheets(file))
        except InvalidFileException as e:
            print(file)
            print(e)
            print('-'*len(str(e)))
        
    return data

In [11]:
df = pd.DataFrame(get_all_data(path_data))

original_data/formularios_03_08/SF/RetomaSP_SF.xls
openpyxl does not support the old .xls file format, please use xlrd to read this file, or convert it to the more recent .xlsx file format.
------------------------------------------------------------------------------------------------------------------------------------------
original_data/formularios_03_08/SEPE/SEPE_REDE_DAORA.xls
openpyxl does not support the old .xls file format, please use xlrd to read this file, or convert it to the more recent .xlsx file format.
------------------------------------------------------------------------------------------------------------------------------------------
original_data/formularios_03_08/PGM/Formul├бrio_Retoma SP desjudicializacao.xls
openpyxl does not support the old .xls file format, please use xlrd to read this file, or convert it to the more recent .xlsx file format.
---------------------------------------------------------------------------------------------------------------------

In [12]:
df.to_excel('dados_extraidos_v1.xlsx')